In [3]:
import importlib
import time
import re
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import ChatGLM
import pandas as pd
import tqdm
import util
import uuid
from typing import Dict,List,Tuple
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter

In [15]:
poetrys = CSVLoader("由大模型处理过的文档/poetrys_72417.csv",encoding="utf-8").load()


In [29]:
new = pd.read_csv("由大模型处理过的文档/poetrys_72417.csv")

,【作者、朝代】,【全文】,【诗题】,【标签】,【赏析、翻译、关于】
0,先秦—佚名,关关雎鸠，在河之洲。窈窕淑女，君子好逑。\r\n参差荇菜，左右流之。窈窕淑女，寤寐求之。\r...,关雎,"['古诗三百首', '初中古诗', '诗经', '爱情']",【翻译】：\n译文\r\n关关和鸣的雎鸠，相伴在河中的小洲。那美丽贤淑的女子，是君子的好配偶...
1,先秦—佚名,遵彼汝坟，伐其条枚。未见君子，惄如调饥。\r\n遵彼汝坟，伐其条肄。既见君子，不我遐弃。\r...,汝坟,"['诗经', '思念']",【翻译】：\n译文\r\n沿着汝河大堤走，采伐山楸那枝条。还没见到我夫君，忧如忍饥在清早。\...
2,先秦—佚名,南山崔崔，雄狐绥绥。鲁道有荡，齐子由归。既曰归止，曷又怀止？\r\n葛屦五两，冠緌双止。鲁道...,南山,"['诗经', '怨刺']",【翻译】：\n译文\r\n南山巍峨高峻，雄狐缓步独行。鲁国大道宽阔，文姜由此嫁人。既然嫁给鲁...
3,唐代—佚名,芳岁肇佳节，物华当仲春。乾坤既昭泰，烟景含氤氲。 \r\n德浅荷玄贶，乐成思治人。前庭列钟鼓...,舞曲歌辞。中和乐舞词,[],【翻译】：\n暂无\n【赏析】：\n暂无\n【关于】：\n暂无
4,唐代—李嘉祐,竹窗松户有佳期，美酒香茶慰所思。辅嗣外生还解易， \r\n惠连群从总能诗。檐前花落春深后，谷...,与从弟正字、从兄兵曹宴集林园,[],【翻译】：\n暂无\n【赏析】：\n暂无\n【关于】：\n暂无
5,唐代—李嘉祐,自顾衰容累玉除，忽承优诏赴铜鱼。江头鸟避青旄节， \r\n城里人迎露网车。长沙地近悲才子，古...,酬皇甫十六侍御曾见寄（此公时贬舒州司马）,[],【翻译】：\n暂无\n【赏析】：\n暂无\n【关于】：\n暂无
6,唐代—李嘉祐,子规夜夜啼槠叶，远道逢春半是愁。芳草伴人还易老， \r\n落花随水亦东流。山临睥睨恒多雨，地...,暮春宜阳郡斋愁坐，忽枉刘七侍御新诗，因以酬答,[],【翻译】：\n暂无\n【赏析】：\n暂无\n【关于】：\n暂无
7,唐代—李嘉祐,诸谢偏推永嘉守，三何独许水曹郎。老兄鄙思难俦匹， \r\n令弟清词堪比量。叠嶂入云藏古寺，高...,送舍弟,[],【翻译】：\n暂无\n【赏析】：\n暂无\n【关于】：\n暂无
8,唐代—李嘉祐,一官万里向千溪，水宿山行鱼浦西。日晚长烟高岸近， \r\n天寒积雪远峰低。芦花渚里鸿相叫，苦...,送从弟永任饶州录事参军,[],【翻译】：\n暂无\n【赏析】：\n暂无\n【关于】：\n暂无
9,唐代—李嘉祐,吴门别后蹈沧州，帝里相逢俱白头。自叹马卿常带病， \r\n还嗟李广未封侯。棠梨宫里瞻龙衮，细...,送马将军奏事毕归滑州使幕,[],【翻译】：\n暂无\n【赏析】：\n暂无\n【关于】：\n暂无


In [35]:
spliter1 = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap  = 64,
    length_function = len,
    is_separator_regex = True,
    separators=[u"【[\u4e00-\u9fa5]{2}】","\n\n",""]
)
spliter2 = RecursiveCharacterTextSplitter(
    chunk_size = 128,
    chunk_overlap  = 16,
    length_function = len,
    is_separator_regex = False,
)

In [56]:
res = []
for i,row in tqdm.tqdm(new.iterrows(),total=len(new)):
    head2 = ("\n标签："+row["【标签】"] if row["【标签】"] != "[]" else "标签：无") +"\n"
    head1 = row["【作者、朝代】"]+"："+"《"+row["【诗题】"]+"》\n"
    head = head1
    res.append(head1+head2)
    contents = spliter2.split_text(row["【全文】"])
    res.extend(head+f"全文{i}："+c for i,c in enumerate(contents))
    docs = spliter1.split_text(row["【赏析、翻译、关于】"])
    res.extend(head+f"赏析翻译片段{i}："+d for i,d in enumerate(docs))   

100%|██████████| 72417/72417 [00:19<00:00, 3705.11it/s]


In [57]:
pd.DataFrame({"文档":res}).to_csv("诗词赏析文档.csv")

In [26]:
docs = spliter1.split_documents(poetrys)

In [27]:
for i in range(10):
    print("===Sep===")
    print(docs[i].page_content)

===Sep===
【作者、朝代】: 先秦—佚名
【全文】: 关关雎鸠，在河之洲。窈窕淑女，君子好逑。
参差荇菜，左右流之。窈窕淑女，寤寐求之。
求之不得，寤寐思服。悠哉悠哉，辗转反侧。
参差荇菜，左右采之。窈窕淑女，琴瑟友之。
参差荇菜，左右芼之。窈窕淑女，钟鼓乐之。
【诗题】: 关雎
【标签】: ['古诗三百首', '初中古诗', '诗经', '爱情']
【赏析、翻译、关于】:
===Sep===
【翻译】：
译文
关关和鸣的雎鸠，相伴在河中的小洲。那美丽贤淑的女子，是君子的好配偶。
参差不齐的荇菜，从左到右去捞它。那美丽贤淑的女子，醒来睡去都想追求她。
追求却没法得到，白天黑夜便总思念她。长长的思念哟，叫人翻来覆去难睡下。
参差不齐的荇菜，从左到右去采它。那美丽贤淑的女子，奏起琴瑟来亲近她。
参差不齐的荇菜，从左到右去拔它。那美丽贤淑的女子，敲起钟鼓来取悦她。

注释
⑴关关：象声词，雌雄二鸟相互应和的叫声。雎鸠（jū jiū）：一种水鸟名，即王鴡。
⑵洲：水中的陆地。
⑶窈窕（yǎo tiǎo）淑女：贤良美好的女子。窈窕，身材体态美好的样子。窈，深邃，喻女子心灵美；窕，幽美，喻女子仪表美。淑，好，善良。
⑷好逑（hǎo qiú）：好的配偶。逑，“仇”的假借字，匹配。
⑸参差：长短不齐的样子。荇（xìng）菜：水草类植物。圆叶细茎，根生水底，叶浮在水面，可供食用。
⑹左右流之：时而向左、时而向右地择取荇菜。这里是以勉力求取荇菜，隐喻“君子”努力追求“淑女”。流，义同“求”，这里指摘取。之：指荇菜。
⑺寤寐（wù mèi）：醒和睡。指日夜。寤，醒觉。寐，入睡。又，马瑞辰《毛诗
===Sep===
淑女”。流，义同“求”，这里指摘取。之：指荇菜。
⑺寤寐（wù mèi）：醒和睡。指日夜。寤，醒觉。寐，入睡。又，马瑞辰《毛诗传笺注通释》说：“寤寐，犹梦寐。”也可通。
⑻思服：思念。服，想。 《毛传》：“服，思之也。”
⑼悠哉（yōu zāi）悠哉：意为“悠悠”，就是长。这句是说思念绵绵不断。悠，感思。见《尔雅·释诂》郭璞注。哉，语气助词。悠哉悠哉，犹言“想念呀，想念呀”。
⑽辗转反侧：翻覆不能入眠。辗，古字作展。展转，即反侧。反侧，犹翻覆。
⑾琴瑟友之：弹琴鼓瑟来亲近她。琴、瑟，皆弦乐器。琴五或七弦，瑟二十五或五十弦。友：用作动词，此处有亲近之意。这句说，用琴瑟来亲